In [1]:
from pathlib import Path
from nltk.corpus import stopwords
import tempfile
import zipfile
import os
import nltk
import re
from os import path
from nltk import word_tokenize
import pandas as pd
import pickle
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Extract the Processed Files to load the files in the Master_Data Folder as a  Destination folder.

In [4]:
new_dir ='Master_Data'

In [5]:
src_path = '/content/drive/MyDrive/IR/Assignment/Processed_Data.zip'
parent_path = '/content/drive/MyDrive'

In [6]:
dest_path = os.path.join(parent_path,new_dir)

In [7]:
if path.exists(dest_path) == False:
  os.mkdir(dest_path)
  with zipfile.ZipFile(src_path,'r') as zip_temp:
     zip_temp.extractall(dest_path)

In [8]:
print(os.getcwd())

/content


In [9]:
os.chdir(dest_path)

2. Get the list of path of files in file_paths_list and the name of all files in file_name_list.

In [10]:
root = dest_path

In [11]:
root = os.getcwd()

In [12]:
file_paths_list = []
file_name_list = []

In [13]:
#path = '/content/drive/MyDrive/IR/Assignment/Processed_Data.zip'
#my_zf = zipfile.ZipFile(path,'r')
#with tempfile.TemporaryDirectory() as Master_Data:
  #my_zf.extractall(Master_Data)
  #root = os.listdir(Master_Data)
for s_dir,dir,files in os.walk(root):
  for f in files:
    file_path = s_dir +'/' + f
    if f.__contains__('cranfield'):
      file_name_list.append(f)
      file_paths_list.append(file_path)

3. Check the total no. of files in the given document and sort the lists, so as when the lists are iterated over, the DOC-ID Postings

In [14]:
file_paths_list.sort()
#print(file_paths_list)

In [15]:
file_name_list.sort()
#print(file_name_list)

In [16]:
print("The total no. of files is as:", len(file_paths_list))

The total no. of files is as: 1400


In [17]:
file_content = []

In [18]:
c = 1
for filepath in file_paths_list:
    with open(filepath, mode='r') as curr_file:
        #file_content += [curr_file.read()]
        #print("Processing for DOC :",c)
        file_content.append(curr_file.read())
        curr_file.close()
        c = c+1

In [ ]:
#print(file_content)

['experimental investigation aerodynamics wing slipstream experimental study wing propeller slipstream made order determine spanwise distribution lift increase due slipstream different angles attack wing different free stream slipstream velocity ratios results intended part evaluation basis different theoretical treatments problem comparative span loading curves together supporting evidence showed substantial part lift increment produced slipstream due destalling boundarylayercontrol effect integrated remaining lift increment subtracting destalling lift found agree well potential flow theory empirical evaluation destalling effects made specific configuration experiment', 'simple shear flow past flat plate incompressible fluid small viscosity study highspeed viscous flow past twodimensional body usually necessary consider curved shock wave emitting nose leading edge body consequently exists inviscid rotational flow region shock wave boundary layer situation arises instance study hyperso

4. Create a dictionary to add DOC-ID and DOC-NAME for each file to identify the file used for the query and then retrieve the corresponding DOC-NAME and DOC-ID for specified output format.

In [19]:
# Initalize an empty Dictionary
# The Dictionary will be used as lookup to get the DOC-ID and DOC-NAME to get the NAME of the documents retrieved for the Query.
def make_Dictionary(file_name_list,file_paths_list):
    D = {}
    d_id = 1
    for i in range(len(file_paths_list)):
      D[file_paths_list[i]] = []
      D[file_paths_list[i]].append(d_id)
      D[file_paths_list[i]].append(file_name_list[i]+'.txt')
      d_id = d_id + 1
      i = i + 1
    return D

In [20]:
# Helper Functions to get DOC-ID and DOC-NAME for a given DOC-PATH. The details are reffered to as DOC_Det
def get_DOC_Det(my_Dict,curr_file_path):
  return my_Dict[curr_file_path]


In [21]:
my_Dict = make_Dictionary(file_name_list,file_paths_list)
print(my_Dict)

{'/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0001': [1, 'cranfield0001.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0002': [2, 'cranfield0002.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0003': [3, 'cranfield0003.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0004': [4, 'cranfield0004.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0005': [5, 'cranfield0005.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0006': [6, 'cranfield0006.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0007': [7, 'cranfield0007.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0008': [8, 'cranfield0008.txt'], '/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0009': [9, 'cranfield000

In [22]:
# Run for an example to check if the values are retrieved or not for a given file path from the Dictionary
file_det = get_DOC_Det(my_Dict,'/content/drive/MyDrive/Master_Data/content/CSE508_Winter2023_Dataset/cranfield0002')
# Get the Details of the file
print("The DOC-ID is as : ",file_det[0])
print("The DOC-NAME is as : ",file_det[1])

The DOC-ID is as :  2
The DOC-NAME is as :  cranfield0002.txt


5. Creating a Unigram Inverted Index by iterating over token of words and appending the DOC-ID corresponding to each word. 
> The desired text for each file is available in the list of file_content and hence we tokenize each file by iterating over file_content.

In [23]:
# Print the content of 'text' in first ten files. There is a need to tokenize the content of the file to get each word.
file_itr = 0
for file_itr in range(10):
  text = file_content[file_itr].strip()
  print(text)
  file_itr = file_itr + 1

experimental investigation aerodynamics wing slipstream experimental study wing propeller slipstream made order determine spanwise distribution lift increase due slipstream different angles attack wing different free stream slipstream velocity ratios results intended part evaluation basis different theoretical treatments problem comparative span loading curves together supporting evidence showed substantial part lift increment produced slipstream due destalling boundarylayercontrol effect integrated remaining lift increment subtracting destalling lift found agree well potential flow theory empirical evaluation destalling effects made specific configuration experiment
simple shear flow past flat plate incompressible fluid small viscosity study highspeed viscous flow past twodimensional body usually necessary consider curved shock wave emitting nose leading edge body consequently exists inviscid rotational flow region shock wave boundary layer situation arises instance study hypersonic v

The Unigram-Inverted Index consists of all unique words as the Key and the corresponding DOC-IDs, if the word exists in a DOC is set as the Value.
> The Key-Value Pair consists of the first index of the value at '0' as the length of the DOC-IDs corresponding to the number of elements in the set.

> The next index has the set of all DOC-IDs, whereever a particular word exists.

In [104]:
# To create a Unigram Inverted Index, we use a  Dictionary
# NOTE - We traverse over the list of file_paths as there is a need to read each file, tokenize the word of file, find unique token and add the DOC_ID corresponding to each token.
# As the DOC-ID and DOC-NAME are corresponding to the path of t
# If the unique word already exists as a UNIGRAM of the 
def make_uni_inv_idx(file_paths_list,my_Dict):
      INV_IDX = {}
      DOC_NAME = {}
      file_itr = 0
      doc = 0
      for filepath in file_paths_list:
        with open(filepath, mode='r') as curr_file:
          print("Processing for DOC :",doc)
          text = (curr_file.read()).strip()
          text_tokens = word_tokenize(text)
          for token in text_tokens:
            # Iterate over the given list of tokens through 'token' as iterator and check if token exists as a key or not.
            target_token = token
            if(target_token in INV_IDX.keys()):
              DOC_det = get_DOC_Det(my_Dict,filepath)
              # Add the DOC_ID
              DOC_NAME[DOC_det[0]] = DOC_det[1]
              # Store the DOC-NAME for the given DOC-ID in a distinct document
              # DOC_NAME[DOC_det[0]] = DOC_det[1]
              # Append the Document-ID
              INV_IDX[target_token][1].add(DOC_det[0])
              # Update the frequency or count of documents.
              INV_IDX[target_token][0] = len(INV_IDX[target_token][1])
            else:
              # As there does not already exists a token corresponding
              DOC_det = get_DOC_Det(my_Dict,filepath)
              # Add the DOC_ID
              DOC_NAME[DOC_det[0]] = DOC_det[1]

              INV_IDX[token] = [1, {DOC_det[0]}]

          #print(text_tokens)
          doc = doc + 1
      return INV_IDX,DOC_NAME

In [105]:
INV_IDX,DOC_NAME = make_uni_inv_idx(file_paths_list,my_Dict)

Processing for DOC : 0
Processing for DOC : 1
Processing for DOC : 2
Processing for DOC : 3
Processing for DOC : 4
Processing for DOC : 5
Processing for DOC : 6
Processing for DOC : 7
Processing for DOC : 8
Processing for DOC : 9
Processing for DOC : 10
Processing for DOC : 11
Processing for DOC : 12
Processing for DOC : 13
Processing for DOC : 14
Processing for DOC : 15
Processing for DOC : 16
Processing for DOC : 17
Processing for DOC : 18
Processing for DOC : 19
Processing for DOC : 20
Processing for DOC : 21
Processing for DOC : 22
Processing for DOC : 23
Processing for DOC : 24
Processing for DOC : 25
Processing for DOC : 26
Processing for DOC : 27
Processing for DOC : 28
Processing for DOC : 29
Processing for DOC : 30
Processing for DOC : 31
Processing for DOC : 32
Processing for DOC : 33
Processing for DOC : 34
Processing for DOC : 35
Processing for DOC : 36
Processing for DOC : 37
Processing for DOC : 38
Processing for DOC : 39
Processing for DOC : 40
Processing for DOC : 41
Pr

In [106]:
print(INV_IDX)
#print(DOC_NAME)

{'experimental': [316, {1, 11, 12, 17, 19, 25, 29, 30, 35, 41, 47, 52, 53, 58, 69, 70, 74, 78, 84, 99, 101, 103, 112, 115, 121, 123, 137, 140, 142, 154, 156, 168, 170, 171, 173, 176, 179, 183, 184, 186, 187, 188, 189, 191, 195, 197, 202, 203, 206, 207, 212, 216, 220, 222, 225, 227, 230, 234, 245, 251, 256, 257, 262, 271, 273, 277, 282, 283, 286, 294, 295, 304, 307, 329, 330, 334, 338, 339, 344, 345, 346, 347, 354, 360, 369, 370, 372, 377, 397, 409, 411, 413, 418, 420, 421, 423, 427, 435, 439, 441, 442, 453, 455, 462, 464, 467, 484, 494, 496, 497, 498, 501, 503, 504, 505, 511, 518, 520, 522, 536, 540, 544, 549, 552, 553, 558, 563, 567, 569, 572, 576, 588, 595, 600, 606, 610, 632, 634, 635, 636, 644, 645, 649, 662, 663, 666, 670, 675, 678, 679, 685, 688, 689, 694, 704, 712, 713, 717, 720, 728, 729, 739, 740, 743, 753, 760, 766, 767, 772, 781, 790, 801, 802, 806, 816, 820, 823, 825, 827, 830, 836, 844, 845, 846, 847, 856, 857, 858, 863, 866, 867, 869, 878, 881, 887, 891, 907, 911, 912, 92

In [107]:
# Check the length of dictionary.
dictLen = len(INV_IDX)

In [108]:
print(dictLen)

8967


6. Create Pickle Files

In [109]:
# The pickle file is in the current working directory.
with open('Unigram-Postings.pickle', 'wb') as curr:
    pickle.dump(INV_IDX, curr, protocol=pickle.HIGHEST_PROTOCOL)


In [110]:
# The pickle file is in the current working directory.
with open('DOC-ID-to-DOC-Name.pickle', 'wb') as curr:
    pickle.dump(DOC_NAME, curr, protocol=pickle.HIGHEST_PROTOCOL)


Use the pickle files from here.

In [111]:
with open('Unigram-Postings.pickle', 'rb') as curr:
     INV_IDX_pckl = pickle.load(curr)

In [112]:

with open('DOC-ID-to-DOC-Name.pickle', 'rb') as curr:
     DOC_NAME_pckl = pickle.load(curr)

In [113]:
print(INV_IDX_pckl)

{'experimental': [316, {1, 11, 12, 17, 19, 25, 29, 30, 35, 41, 47, 52, 53, 58, 69, 70, 74, 78, 84, 99, 101, 103, 112, 115, 121, 123, 137, 140, 142, 154, 156, 168, 170, 171, 173, 176, 179, 183, 184, 186, 187, 188, 189, 191, 195, 197, 202, 203, 206, 207, 212, 216, 220, 222, 225, 227, 230, 234, 245, 251, 256, 257, 262, 271, 273, 277, 282, 283, 286, 294, 295, 304, 307, 329, 330, 334, 338, 339, 344, 345, 346, 347, 354, 360, 369, 370, 372, 377, 397, 409, 411, 413, 418, 420, 421, 423, 427, 435, 439, 441, 442, 453, 455, 462, 464, 467, 484, 494, 496, 497, 498, 501, 503, 504, 505, 511, 518, 520, 522, 536, 540, 544, 549, 552, 553, 558, 563, 567, 569, 572, 576, 588, 595, 600, 606, 610, 632, 634, 635, 636, 644, 645, 649, 662, 663, 666, 670, 675, 678, 679, 685, 688, 689, 694, 704, 712, 713, 717, 720, 728, 729, 739, 740, 743, 753, 760, 766, 767, 772, 781, 790, 801, 802, 806, 816, 820, 823, 825, 827, 830, 836, 844, 845, 846, 847, 856, 857, 858, 863, 866, 867, 869, 878, 881, 887, 891, 907, 911, 912, 92

In [114]:
print(DOC_NAME_pckl)

{1: 'cranfield0001.txt', 2: 'cranfield0002.txt', 3: 'cranfield0003.txt', 4: 'cranfield0004.txt', 5: 'cranfield0005.txt', 6: 'cranfield0006.txt', 7: 'cranfield0007.txt', 8: 'cranfield0008.txt', 9: 'cranfield0009.txt', 10: 'cranfield0010.txt', 11: 'cranfield0011.txt', 12: 'cranfield0012.txt', 13: 'cranfield0013.txt', 14: 'cranfield0014.txt', 15: 'cranfield0015.txt', 16: 'cranfield0016.txt', 17: 'cranfield0017.txt', 18: 'cranfield0018.txt', 19: 'cranfield0019.txt', 20: 'cranfield0020.txt', 21: 'cranfield0021.txt', 22: 'cranfield0022.txt', 23: 'cranfield0023.txt', 24: 'cranfield0024.txt', 25: 'cranfield0025.txt', 26: 'cranfield0026.txt', 27: 'cranfield0027.txt', 28: 'cranfield0028.txt', 29: 'cranfield0029.txt', 30: 'cranfield0030.txt', 31: 'cranfield0031.txt', 32: 'cranfield0032.txt', 33: 'cranfield0033.txt', 34: 'cranfield0034.txt', 35: 'cranfield0035.txt', 36: 'cranfield0036.txt', 37: 'cranfield0037.txt', 38: 'cranfield0038.txt', 39: 'cranfield0039.txt', 40: 'cranfield0040.txt', 41: 'cra

7. Defining the operations to execute complex Boolean Queries.
Note :- The DOC-IDs are stored in a set in the generated Unigram Inverted Index, to avoid repetition, in case if a word exists at multiple positions in the given document. However while processing we convert each of the set to lists and use the algoritm discussed for merging.

# 7.1 OR Queries

In [115]:
# OR - Merge two lists using the OR - Operation
# Return the count of operations and the list of appropriate DOC_IDs
def OR(s1,s2):
  # opt_count has the minimum no. of operations required
  opt_count = 0
  # The INV-IDX has the No. of Documents, corresponding to each term and is used here as the length of the two lists to be merged.
  len1 = s1[0]
  len2 = s2[0]
  lst1 = list(s1[1])
  lst2 = list(s2[1])
  # SORT - To optimise the no. of operations
  lst1.sort()
  lst2.sort()
  # Initialize a res list to keep the resultant DOC_IDs
  res = []
  #res = set(lst1 + lst2)
  #res = list(res)
  i = 0
  j = 0
  while(i<len1 and j<len2):
    if(lst1[i] == lst2[j]):
      doc_id = lst1[i]
      res.append(doc_id)
      i = i+1
      j = j+1
      opt_count = opt_count + 1;
    else:
      # For OR, the word can either exist in List1 or in List2
      if(lst1[i]<lst2[j]):
        doc_id = lst1[i]
        res.append(doc_id)
        i = i+1
        opt_count = opt_count + 1;
      else:
        doc_id = lst2[j]
        res.append(doc_id)
        j = j+1
        opt_count = opt_count + 1;
  # Append the longer list now and no comparisons are required.
  while(i < len1):
    doc_id = lst1[i]
    res.append(doc_id)
    i = i+1
  while(j<len2):
    doc_id = lst2[j]
    res.append(doc_id)
    j = j+1
  return res,opt_count

# 7.2 AND Queries

In [53]:
def AND(s1,s2):
  print('AND is here.')
  # opt_count has the minimum no. of operations required
  opt_count = 0
  # The INV-IDX has the No. of Documents, corresponding to each term and is used here as the length of the two lists to be merged.
  len1 = s1[0]
  len2 = s2[0]
  lst1 = list(s1[1])
  lst2 = list(s2[1])
  # SORT - To optimise the no. of operations
  lst1.sort()
  lst2.sort()
  #print(lst1)
  #print(lst2)
  # Initialize a res list to keep the resultant DOC_IDs
  res = []
  i = 0
  j = 0
  while(i<len1 and j<len2):
    if(lst1[i] == lst2[j]):
      doc_id = lst1[i]
      res.append(doc_id)
      i = i+1
      j = j+1
      opt_count = opt_count + 1
    else:
      # For AND, the DOC-ID must exist in List1 and in List2
      if(lst1[i]<lst2[j]):
        i = i+1
        opt_count = opt_count + 1
      else:
        j = j+1
        opt_count = opt_count + 1
  return res,opt_count

# 7.3 AND NOT Queries

In [37]:
def AND_NOT(s1,s2):
  # opt_count has the minimum no. of operations required
  opt_count = 0
  # The INV-IDX has the No. of Documents, corresponding to each term and is used here as the length of the two lists to be merged.
  len1 = s1[0]
  len2 = s2[0]
  lst1 = list(s1[1])
  lst2 = list(s2[1])
  # SORT - To optimise the no. of operations
  lst1.sort()
  lst2.sort()
  # Initialize a res list to keep the resultant DOC_IDs
  res = []
  i = 0
  j = 0
  while(i<len1 and j<len2):
      # For AND NOT, the DOC-ID which exist in List1 and do not exist in List2, will definately exist in complement of List2 and hence such words are added to result.
      if(lst1[i]<lst2[j]):
        doc_id = lst1[i]
        res.append(doc_id)
        i = i+1
        opt_count = opt_count + 1;
      # Rule out the case for when the DOC-ID exists in List-2
      elif(lst1[i] == lst2[j]):
        i = i+1
        j = j+1
      # Rule out the case for when the DOC-ID exists in List-1 and List-2
      else:
        j = j+1
        opt_count = opt_count + 1;
  return res,opt_count

# 7.4 OR NOT Queries

In [80]:
def OR_NOT(s1,s2):
  # opt_count has the minimum no. of operations required
  opt_count = 0
  # The INV-IDX has the No. of Documents, corresponding to each term and is used here as the length of the two lists to be merged.
  len1 = s1[0]
  len2 = s2[0]
  lst1 = list(s1[1])
  lst2 = list(s2[1])
  # SORT - To optimise the no. of operations
  lst1.sort()
  lst2.sort()
  # Initialize a res list to keep the resultant DOC_IDs
  lst2_new = []
  res = []
  i = 0
  j = 0

  # Take the NOT of the 2nd List as lst2
  for i in range(1400):
    if i+1 in lst2:
      continue
    else:
      lst2_new.append(i+1)

  len2 = len(lst2_new)
  lst2 = lst2_new
  # Initialize a res list to keep the resultant DOC_IDs
  res = []
  i = 0
  j = 0
  while(i<len1 and j<len2):
    if(lst1[i] == lst2[j]):
      doc_id = lst1[i]
      res.append(doc_id)
      i = i+1
      j = j+1
      opt_count = opt_count + 1;
    else:
      # For OR, the word can either exist in List1 or in List2
      if(lst1[i]<lst2[j]):
        doc_id = lst1[i]
        res.append(doc_id)
        i = i+1
        opt_count = opt_count + 1;
      else:
        doc_id = lst2[j]
        res.append(doc_id)
        j = j+1
        opt_count = opt_count + 1;
  # Append the longer list now and no comparisons are required.
  while(i < len1):
    doc_id = lst1[i]
    res.append(doc_id)
    i = i+1
  while(j<len2):
    doc_id = lst2[j]
    res.append(doc_id)
    j = j+1
  return res,opt_count

In [43]:
# Helper Function to execute the Queries and store the intermediate results.
# Returns the count of Documents retrieved, the DOC-IDS
# Each function call will return the list of retrieved documents as well as the count of operations performed so far.
def get_ans(ip_seq_prcsd,ops_list):
  res_count = 0
  doc_id_lists = []
  for word in ip_seq_prcsd:
    if word in INV_IDX_pckl.keys():
      doc_id_lists.append(INV_IDX_pckl[word])
    else:
      # Set length of DOC_ID list to 0 and the DOC_ID_List to NULL or EMPTY.
      doc_id_lists.append(0,{})
    # doc_id_list, now has the list for every possible word.
    # To execute complex queries we begin by considering the first two lists to execute 1st operations and then we iterate over the operation and update the list.
  #print(doc_id_lists)
  lst1 = doc_id_lists[0]
  lst_temp = []
  total_opt_count = 0
  for i in range(0, len(ops_list)):
    lst2 = doc_id_lists[i+1]
    if(ops_list[i] == 'AND'):
      lst_temp,counter = AND(lst1,lst2)
    if(ops_list[i] == 'OR'):
      lst_temp,counter = OR(lst1,lst2)
    if(ops_list[i] == 'OR NOT'):
      lst_temp,counter = OR_NOT(lst1,lst2)
    if(ops_list[i] == 'AND NOT'):
      lst_temp,counter = AND_NOT(lst1,lst2)
    lst1 = lst_temp
    i = i + 1
    total_opt_count += counter
  return lst_temp, total_opt_count


8. Take the Input in the specified format.

Input format:

a. The first line contains N denoting the number of queries to execute

b. The next 2N lines contain queries in the following format:

i. Input sequence
ii. Operations separated by comma

> RIGHT NOW, the ip_seq_prcsd is same as ip_seq.Now we define the corresponding equations with ip_seq

In [40]:
# Helper function to pre-process the INPUT - QUERY
def pre_process(ip_seq):
  input_str = ip_seq
  input_str = input_str.lower()
  input_str = re.sub(r'[^\w\s]', '', input_str).split()
  without_stopwords = []
  for word in input_str:
    if word not in stop_words:
      without_stopwords.append(word)
  return without_stopwords

In [41]:
# Helper function to genrate a complete Query to print it.
def makeQuery(res,opt):
  Query = " "
  for i in range(len(res)):
    Query += res[i]
    Query += " "
    if(i <= (len(opt)-1)):
      Query += opt[i]
      Query += " "
  return Query

In [42]:
def begin():
  # Ask the user to enter the no. of Queries to run the given program for.
  print('Start: ')
  res_dict = {}
  nq = int(input(" Enter the No. of Queries to execute: "))
  queries = []
  for i in range(nq) :
    ip_seq = input(" Enter the input sentence: ")
    ops = input(" Enter the operations: ")
    # Split on the basis of ',', get the list of sequence of operations to be performed.
    ops_list = ops.split(',')
    #print(ops_list)
    # Store the result after processing in a dictionary where the key is the Query No. and the value has the answers.
    # prcsd_query = "PREPROCESS AND JOIN THE INPUT AND SEQUENCE OF OPERATIONS TO GET THE ANS "
    ###################------ADD FUNCTION HERE---------------###################
    ip_seq_prcsd = pre_process(ip_seq)
    queries.append(makeQuery(ip_seq_prcsd,ops_list))
    res_dict[i+1] = get_ans(ip_seq_prcsd,ops_list)
    i = i + 1
  opt_result(nq, res_dict,queries)
  return

9. Display the output/result in the specified format.
a. 4N lines consisting of the results in the following format:

i. Query X

ii. Number of documents retrieved for query X

iii. Names of the documents retrieved for query X

iv. Number of comparisons required for query X

In [84]:
def opt_result(nq, res_dict,queries):
  for i in range(nq):
    print(" The Input Query is :  ",queries[i])
    res_lst = res_dict[i+1]
    print(" The No. of documents retrieved for Query is:",len(res_lst[0]))
    #print(res_lst[0])
    print(" The name of documents for the retrieved Query is:")
    fin_lst = []
    for val in res_lst[0]:
      print(DOC_NAME_pckl[val])
    print("The No. of comparisons executed for the Query is:",res_lst[1])

In [116]:
begin()

Start: 
 Enter the No. of Queries to execute: 1
 Enter the input sentence: experimental flow
 Enter the operations: OR NOT
 The Input Query is :    experimental OR NOT flow 
 The No. of documents retrieved for Query is: 872
 The name of documents for the retrieved Query is:
cranfield0001.txt
cranfield0005.txt
cranfield0008.txt
cranfield0010.txt
cranfield0011.txt
cranfield0012.txt
cranfield0013.txt
cranfield0014.txt
cranfield0015.txt
cranfield0017.txt
cranfield0019.txt
cranfield0020.txt
cranfield0025.txt
cranfield0029.txt
cranfield0030.txt
cranfield0031.txt
cranfield0032.txt
cranfield0035.txt
cranfield0040.txt
cranfield0041.txt
cranfield0042.txt
cranfield0043.txt
cranfield0046.txt
cranfield0047.txt
cranfield0052.txt
cranfield0053.txt
cranfield0058.txt
cranfield0062.txt
cranfield0065.txt
cranfield0066.txt
cranfield0067.txt
cranfield0068.txt
cranfield0069.txt
cranfield0070.txt
cranfield0071.txt
cranfield0074.txt
cranfield0075.txt
cranfield0076.txt
cranfield0077.txt
cranfield0078.txt
cranf